In [1]:
"""def find_diff_flux_for_energy(energy):
    R  = 1.74 * 10 ** 8 # in cm
    L = 10 ** 5
    
    integrand = lambda z, r, theta: diff_muon_flux_per_cubic_cm_moon(find_muon_initial_energy(energy, z)) * r
    z_lower = lambda theta, r: -math.sqrt(R ** 2 - r)
    z_upper = lambda theta, r: L - math.sqrt(R ** 2 - r)
    r_lower = 0.0
    r_upper = math.sqrt(R ** 2 - 0.25 * L ** 2)
    theta_lower = 0
    theta_upper = 2 * np.pi
    
    
    return tplquad(integrand, theta_lower, theta_upper, r_lower, r_upper, z_lower, z_upper)"""

'def find_diff_flux_for_energy(energy):\n    R  = 1.74 * 10 ** 8 # in cm\n    L = 10 ** 5\n    \n    integrand = lambda z, r, theta: diff_muon_flux_per_cubic_cm_moon(find_muon_initial_energy(energy, z)) * r\n    z_lower = lambda theta, r: -math.sqrt(R ** 2 - r)\n    z_upper = lambda theta, r: L - math.sqrt(R ** 2 - r)\n    r_lower = 0.0\n    r_upper = math.sqrt(R ** 2 - 0.25 * L ** 2)\n    theta_lower = 0\n    theta_upper = 2 * np.pi\n    \n    \n    return tplquad(integrand, theta_lower, theta_upper, r_lower, r_upper, z_lower, z_upper)'

In [ ]:
"""  return E_final
def make_adjusted_muon_diff_flux_per_cm3_arrs(initial_muon_energies, distance_through_moon):
    log_muon_final_energies = [] #these need to be log because we will integrate across them eventually
    muon_diff_fluxes = []
    for energy in initial_muon_energies:
        final_energy = find_muon_final_energy(energy, distance_through_moon)
        if final_energy > (10**5): #needs 10^5 GeVs to get to earth, this also has room for improvement!
            log_muon_final_energies.append(np.log10(final_energy))
            muon_diff_fluxes.append(diff_muon_flux_per_cubic_cm_moon(energy))
            
    return log_muon_final_energies, muon_diff_fluxes #flux is /cm^3/GeV
    
    
initial_muon_energies = np.logspace(1, 9, 100)

log_muon_final_energies, muon_diff_fluxes = make_adjusted_muon_diff_flux_per_cm3_arrs(initial_muon_energies, 10**5)

diff_signal_muon_flux_per_area = interp1d(log_muon_final_energies, muon_diff_fluxes)
plt.plot(log_muon_final_energies, muon_diff_fluxes,'k')

plt.xlabel('log10 Signal Muon Energy (GeV)')
plt.ylabel('signal muon flux (cm-3 GeV-1 sec-1)')
plt.yscale('log')
plt.grid(True)    
    """

In [ ]:
#simulates oscillation in a uniform moon
def neutrino_through_uniform_moon(moon_uniform_density, energy): #density in g/cm^3, energy in GeVs
    moon_diameter = 1740 * 2 #km
    after_moon = ThreeFlavorProbs(energy,moon_diameter,np.pi/3,moon_uniform_density,get_prob_amplitudes(energy))#[4][-1] 
    return after_moon

In [ ]:
"""test3 = ThreeFlavorProbs(0.005,200,np.pi/3,0,[1,0,0,0,0,0]) # E = 0.005 GeV = 5 MeV, L = 200 km, delta_13 = pi/3
plt.plot(test3[0],test3[1],label=r'$P_e$')
plt.plot(test3[0],test3[2],label=r'$P_{\mu}$')
plt.plot(test3[0],test3[3],label=r'$P_{\tau}$')
plt.legend(loc='best')
plt.xlabel('L (km)')
plt.ylabel('Probability')
plt.annotate(r'$E_{\nu} = 1$ GeV', xy=(0.5, 1.03), xycoords='axes fraction', ha='center')#, bbox=dict(facecolor='white', alpha=0.8))
#plt.savefig("nu-oscillation-probabilities-3-flavor",dpi=300,bbox_inches='tight')"""

In [ ]:
#energy in GeVs
#densities in g/cm^3, [crust, mantle, partial melt layer, fluid outer core, solid inner core]
#lengths in km
#simulates neutrino oscillation through moon layers of varied densities
def neutrino_trough_layered_moon(energy, densities, lengths): 
    previous_data = []
    distance_traveled = 0
    for i in range(len(densities)):
        if i == 0:
            new_travel_data = ThreeFlavorProbs(energy,lengths[i],np.pi/3,densities[i],get_prob_amplitudes(energy))
            for j in range(len(new_travel_data)):
                previous_data.append(new_travel_data[j])
        else:
            distance_traveled += lengths[i - 1]
            previous_flavor_amplitudes = previous_data[4][-1]
            new_travel_data = ThreeFlavorProbs(energy,lengths[i],np.pi/3,densities[i], previous_flavor_amplitudes)
            
            for k in range(len(new_travel_data[0])):
                new_travel_data[0][k] += distance_traveled

            for j in range(len(previous_data)):
                previous_data[j] = np.concatenate((previous_data[j], new_travel_data[j]))

    return previous_data

In [ ]:
def get_flavor_probabilities_from_core_density(energy, core_density):
    moon_transit = neutrino_trough_layered_moon(energy, [2, 3, core_density, 3, 2], [100, 1160, 2* (150 + 90 + 240), 1160, 100])
    after_transit_to_earth = neutrino_travel_to_earth(energy, moon_transit)
    p_elec = after_transit_to_earth[1][-1]
    p_muon = after_transit_to_earth[2][-1]
    p_tau = after_transit_to_earth[3][-1]
    
    return p_elec, p_muon, p_tau

In [ ]:
def visualize_core_density_vs_probabilities(energy):
    densities = np.linspace(0, 50, 50) #range of core densities to test
    elec_probabilities = []
    muon_probabilities = []
    tau_probabilities = []
    for density in densities:
        new_probabilities = get_flavor_probabilities_from_core_density(energy, density)
        elec_probabilities.append(new_probabilities[0])
        muon_probabilities.append(new_probabilities[1])
        tau_probabilities.append(new_probabilities[2])
        
    plt.plot(densities,elec_probabilities,label=r'$P_e$', linestyle=':')
    plt.plot(densities,muon_probabilities,label=r'$P_{\mu}$', linestyle='-')
    plt.plot(densities,tau_probabilities,label=r'$P_{\tau}$', linestyle='--')
    plt.legend(loc='best')
    plt.xlabel('Moon Core Density (g/cm^3)')
    plt.ylabel('Flavor Probability')
    plt.annotate(r'$E_{\nu} = 1$ GeV', xy=(0.5, 1.03), xycoords='axes fraction', ha='center')#, fontsize=12)
    #plt.savefig("core_density_vs_nu_flavor_updated",dpi=300,bbox_inches='tight')
    return

In [ ]:
"""
def three_flavor_prob_moon_density_ratio(energy, y, z, ratio): #energy in GeVs, y and z in km
    r_moon = 1737 #km
    x_final = np.sqrt(r_moon**2 - y**2 - z**2)
    x0 = -x_final
    
    x = np.linspace(x0,x_final,1000)
    del13 = np.pi/3
    solution = odeint(moon_deriv_three_flavor,get_prob_amplitudes(energy),x,args=(energy,del13,y, z, density_function))
    ProbNuE = solution[:,0]*solution[:,0] + solution[:,1]*solution[:,1]
    ProbNuMu = solution[:,2]*solution[:,2] + solution[:,3]*solution[:,3]
    ProbNuTau = solution[:,4]*solution[:,4] + solution[:,5]*solution[:,5]"""

In [ ]:
no_mass = ThreeFlavorProbs(1,3000,np.pi/3,0,[1, 0, 0, 0, 0, 0]) # E = 0.005 GeV = 5 MeV, L = 200 km, delta_13 = pi/3
mass = ThreeFlavorProbs(1,3000,np.pi/3,3.5,[1, 0, 0, 0, 0, 0])
#plt.plot(test3[0],test3[1],label=r'$P_e$')
plt.plot(no_mass[0],no_mass[1],label=r'Vacuum', linewidth=2)
plt.plot(mass[0],mass[1],label=r'${\rho}=3.5 g/cm^3$', linewidth=2)

#plt.plot(test3[0],test3[3],label=r'$P_{\tau}$')
plt.legend(loc='best')
plt.xlabel('L (km)')
plt.ylabel('Electron Flavor Probability')
plt.annotate(r'$E_{\nu} = 1$ GeV', xy=(0.5, 1.03), xycoords='axes fraction', ha='center')#, bbox=dict(facecolor='white', alpha=0.8))
plt.savefig("slideshow_fig_mass_oscillation",dpi=300,bbox_inches='tight')

In [ ]:
oscillation_example = ThreeFlavorProbs(0.005,400,np.pi/3,0,[1,0,0,0,0,0]) # E = 0.005 GeV = 5 MeV, L = 200 km, delta_13 = pi/3
plt.plot(oscillation_example[0],oscillation_example[1],label=r'$P_e$')
plt.plot(oscillation_example[0],oscillation_example[2],label=r'$P_{\mu}$', color="orange")
plt.plot(oscillation_example[0],oscillation_example[3],label=r'$P_{\tau}$', color="green")
plt.legend(loc='best')
plt.xlabel('L (km)')
plt.ylabel('Probability')
plt.annotate(r'$E_{\nu} = 0.005$ GeV', xy=(0.5, 1.03), xycoords='axes fraction', ha='center')#, bbox=dict(facecolor='white', alpha=0.8))
plt.savefig("nu-oscillation-example",dpi=300,bbox_inches='tight')


In [7]:
import import_ipynb
import lunar_nue_flux_oom

ModuleNotFoundError: No module named 'lunar_nue_flux_oom'

In [ ]:
def deepcore_integrand(energy):
    moon_solid_angle = 0.0000648
    return (a_eff_elec(energy) * e_flux(energy) + a_eff_muon(energy) * m_flux(energy)) * moon_solid_angle


In [ ]:
"""energies = np.linspace(1, 10, 100)
plt.plot(energies, uniform_core[2](energies), label=r'$\rho_{core}=3.5$ g/cm$^3$')
plt.plot(energies, dense_core[2](energies), label=r'$\rho_{core}=10$ g/cm$^3$')
plt.plot(energies, super_dense_core[2](energies), label=r'$\rho_{core}=100$ g/cm$^3$')
plt.legend(loc='best')
#plt.yscale("log")
plt.xlabel('Energy (GeV)')
#plt.ylabel(r'e * $\phi_{\nu_e}$ (m$^{-2}$sr$^{-1}$s$^{-1}$GeV$^{-1}$)')
plt.ylabel(r'Expected Flavor Ratio ($\nu_e:\nu_{\mu}) $')
plt.annotate(r'$R_{core} = 330$ km', xy=(0.5, 1.03), xycoords='axes fraction',
             ha='center')  #, bbox=dict(facecolor='white', alpha=0.8))
plt.savefig("slideshow_flavor_ratio_by_density", dpi=300, bbox_inches='tight')"""

In [ ]:
"""energies = np.linspace(1, 10, 100)
plt.plot(energies, usual_radius[2](energies), label=r'$R_{core} = 300$ km')
plt.plot(energies, bigger_radius[2](energies), label=r'$R_{core} = 400$ km')
plt.plot(energies, much_bigger_radius[2](energies), label=r'$R_{core} = 700$ km')
plt.legend(loc='best')
#plt.yscale("log")
plt.xlabel('Energy (GeV)')
#plt.ylabel(r'$\phi_{\nu_\mu}$ (m$^{-2}$sr$^{-1}$s$^{-1}$GeV$^{-1}$)')
plt.ylabel(r'Expected Flavor Ratio ($\nu_e:\nu_{\mu}) $')
plt.annotate(r'$\rho_{core}=10$ g/cm$^3$', xy=(0.5, 1.03), xycoords='axes fraction',
             ha='center')  #, bbox=dict(facecolor='white', alpha=0.8))
plt.savefig("slideshow_flavor_ratio_by_radius", dpi=300, bbox_inches='tight')"""